![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# **_Análisis de Segmentación de Pisos mediante Clustering🏡_**

## **_Introducción_**
En este notebook, realizaremos un análisis de clustering para segmentar los pisos en función de sus características y la influencia de su entorno. El objetivo es identificar patrones en el mercado inmobiliario y agrupar propiedades con similitudes en términos de ubicación, tipo de negocios cercanos, acceso a transporte y otras variables relevantes.


## **_Objetivos del análisis_**

Segmentación del mercado inmobiliario: Identificar grupos de pisos con precios y características similares.
Evaluar la influencia del entorno: Analizar cómo la cercanía a diferentes tipos de negocios, transporte público y otros factores afectan los precios de los inmuebles.
Detección de zonas con potencial de valorización: Identificar áreas en crecimiento o con características similares a zonas más caras.
Optimización de estrategias de inversión: Proporcionar información para la toma de decisiones en compra, venta o inversión en bienes raíces.


## **_Metodología_**

##### **_1. Carga y exploración de datos:_**

Revisión de los datos disponibles .
Identificación, tratamiento de valores nulos o inconsistencias, entre otras acciones.

##### **_2. Preprocesamiento de datos:_**

Codificación de variables categóricas como barrio y otras en relacion a negocios y la vivienda .
Normalización de variables numéricas (precios, distancias, etc.).

-------------------------------------------------------------------------------------------------------------------
**_Los primeros dos pasos fueron realizados en el notebook donde esta el registro del EDA_** 

-------------------------------------------------------------------------------------------------------------------


##### **_3. Selección de variables:_**

Elección de las características más relevantes para el análisis.
Análisis de correlación entre características y precio.

##### **_4. Aplicación de Clustering:_**

Prueba de diferentes algoritmos de clustering (K-Means, DBSCAN, GMM, etc.).
Selección del número óptimo de clusters (método del codo, silhouette score, etc.).
Evaluación de los resultados y ajuste de parámetros si es necesario.

##### **_5. Análisis e interpretación de clusters:_**

Visualización de los clusters(posiblemente un mapa).
Comparación de los precios y características dentro de cada cluster.
Evaluación del impacto de los negocios, el transporte, entre otras variables en los precios.

##### **_6. Aplicación de clustering en modelos predictivos:_**

Uso de los clusters como una nueva variable en modelos de regresión para mejorar la predicción de precios.
Evaluación del impacto de cada cluster en el precio de los pisos.

**_Conclusión esperada_**

Este análisis proporcionará una segmentación detallada del mercado inmobiliario, ayudando a identificar zonas con potencial de inversión y a entender mejor los factores que influyen en los precios de los inmuebles. Además, permitirá mejorar modelos de predicción de precios al incluir la influencia del entorno en los resultados.



# **_Importacion librerias_**

In [1]:
import pandas as pd 
import numpy as np

# visuzalizaciones
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots


# **_Lectura de datos_**

In [ ]:
df_clustering = pd.read_csv("./data/")

# **_Ajustes extras_**

In [ ]:
pd.set_option("display_max_columns", None)
pd.reset_option("display_max_columns", None)
pd.set_option("display_max_rows", None)
pd.reset_option("display_max_rows", None)

OptionError: "No such keys(s): 'display_max_columns'"

# **_Modelado Clustering_**

### **_Seleccion de variables_**



### **_Aplicacion de Clustering_**

### **_Analisis e interpretacion de clusters_**

Aplicacion de 

# **_Conclusiones finales_**

Despues de la creacion de los anteriores modelos se llega a la conclusion que el mejor modelo es XXXX con un resultado de XXXX y que gracias a el mismo podemos entender que sucede lo siguiente....